<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Cloud Computing Data Science</font>
## <font color='blue'>Projeto 4</font>
### <font color='blue'>Construção de Feature Store com SageMaker e Apache Iceberg</font>

In [1]:
# Imports
import sys
import time
import boto3
import subprocess
import sagemaker
import importlib
import logging
import pandas as pd
import numpy as np
from time import gmtime, strftime, sleep
from random import randint
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.inputs import TableFormatEnum

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Extraindo a versão do pacote sagemaker
sm_version = sagemaker.__version__

In [3]:
print(sm_version)

2.227.0


In [4]:
# Extraindo os releases e pacthes
major, minor, patch = sm_version.split('.')

In [5]:
# Verifica a condição e instala a versão mínima do sagemaker (se necessário)
if int(major) < 2 or int(minor) < 125:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker==2.125.0'])
    importlib.reload(sagemaker)

In [6]:
# Configura um logger em Python para facilitar o registro de mensagens de log 
# em diferentes níveis (como DEBUG, INFO, WARNING, ERROR, e CRITICAL)
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [7]:
# Versões dos pacotes principais
logger.info(f'Versão do SageMaker: {sagemaker.__version__}')
logger.info(f'Versão do Pandas: {pd.__version__}')
logger.info(f'Versão do Numpy: {np.__version__}')
logger.info(f'Versão do Boto3: {boto3.__version__}')

Versão do SageMaker: 2.227.0
Versão do Pandas: 2.2.3
Versão do Numpy: 1.26.4
Versão do Boto3: 1.34.162


## Sessão SageMaker e Definição da Infraestrutura

In [8]:
# Define a sessão
sagemaker_session = sagemaker.Session()

In [9]:
# Captura a role com o privilégio de execução do SageMaker
role = sagemaker.get_execution_role()

In [10]:
print(role)

arn:aws:iam::890582101704:role/service-role/AmazonSageMaker-ExecutionRole-20240523T163775


In [11]:
# Define o Bucket padrão
default_bucket = sagemaker_session.default_bucket()

In [12]:
# Logger
logger.info(f'S3 Bucket Default = {default_bucket}')

S3 Bucket Default = sagemaker-us-east-2-890582101704


In [13]:
# Nome para a feature store
prefix = 'sagemaker-feature-store-p4'

In [14]:
# Captura a região
region = sagemaker_session.boto_region_name

In [15]:
print(region)

us-east-2


## Carga e Preparação dos Dados

In [16]:
# Carrega o dataset de clientes
df_dsa_clientes = pd.read_csv('dados/clientes.csv')

In [17]:
df_dsa_clientes.head()

,id_cliente,genero,casado,event_time,idade_18-29,idade_30-39,idade_40-49,idade_50-59,idade_60-69,idade_70-plus,num_dias_ativo
0,C1,0,0,2024-11-27T04:57:49.698Z,False,False,False,True,False,False,0.083562
1,C2,1,0,2024-11-27T04:57:49.702Z,True,False,False,False,False,False,0.659589
2,C3,1,1,2024-11-27T04:57:49.708Z,False,False,False,False,True,False,0.402055
3,C4,1,0,2024-11-27T04:57:49.712Z,False,False,False,True,False,False,0.708904
4,C5,1,1,2024-11-27T04:57:49.719Z,False,True,False,False,False,False,0.765753


In [18]:
df_dsa_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_cliente      10000 non-null  object 
 1   genero          10000 non-null  int64  
 2   casado          10000 non-null  int64  
 3   event_time      10000 non-null  object 
 4   idade_18-29     10000 non-null  bool   
 5   idade_30-39     10000 non-null  bool   
 6   idade_40-49     10000 non-null  bool   
 7   idade_50-59     10000 non-null  bool   
 8   idade_60-69     10000 non-null  bool   
 9   idade_70-plus   10000 non-null  bool   
 10  num_dias_ativo  10000 non-null  float64
dtypes: bool(6), float64(1), int64(2), object(2)
memory usage: 449.3+ KB


Essa linha abaixo ajusta os tipos de dados das colunas no DataFrame df_dsa_clientes de forma automática e otimizada, com as seguintes características:

- infer_objects=True: Tenta inferir tipos mais específicos para colunas de tipo object (por exemplo, converte strings em categorias, se possível).
- convert_boolean=False: Mantém as colunas booleanas como estão, sem convertê-las para outros tipos.
- convert_dtypes: Ajusta os tipos para versões mais eficientes ou apropriadas (como Int64 em vez de int padrão).

In [19]:
# Ajusta os tipos de dados de forma automática
df_dsa_clientes = df_dsa_clientes.convert_dtypes(infer_objects=True, convert_boolean=False)

In [20]:
df_dsa_clientes.dtypes

id_cliente        string[python]
genero                     Int64
casado                     Int64
event_time        string[python]
idade_18-29                Int64
idade_30-39                Int64
idade_40-49                Int64
idade_50-59                Int64
idade_60-69                Int64
idade_70-plus              Int64
num_dias_ativo           Float64
dtype: object

In [21]:
# Ajusta o tipo de dado como string de forma explícita
df_dsa_clientes['id_cliente'] = df_dsa_clientes['id_cliente'].astype('string')
df_dsa_clientes['event_time'] = df_dsa_clientes['event_time'].astype('string')

In [22]:
df_dsa_clientes.dtypes

id_cliente        string[python]
genero                     Int64
casado                     Int64
event_time        string[python]
idade_18-29                Int64
idade_30-39                Int64
idade_40-49                Int64
idade_50-59                Int64
idade_60-69                Int64
idade_70-plus              Int64
num_dias_ativo           Float64
dtype: object

In [23]:
df_dsa_clientes.head()

,id_cliente,genero,casado,event_time,idade_18-29,idade_30-39,idade_40-49,idade_50-59,idade_60-69,idade_70-plus,num_dias_ativo
0,C1,0,0,2024-11-27T04:57:49.698Z,0,0,0,1,0,0,0.083562
1,C2,1,0,2024-11-27T04:57:49.702Z,1,0,0,0,0,0,0.659589
2,C3,1,1,2024-11-27T04:57:49.708Z,0,0,0,0,1,0,0.402055
3,C4,1,0,2024-11-27T04:57:49.712Z,0,0,0,1,0,0,0.708904
4,C5,1,1,2024-11-27T04:57:49.719Z,0,1,0,0,0,0,0.765753


In [24]:
# Carrega o dataset de produtos
df_dsa_produtos = pd.read_csv('dados/produtos.csv')

In [25]:
df_dsa_produtos.head()

,id_produto,event_time,category_baby_food_formula,category_baking_ingredients,category_candy_chocolate,category_chips_pretzels,category_cleaning_products,category_coffee,category_cookies_cakes,category_crackers,...,category_hair_care,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt
0,P1,2024-11-27T05:02:13.726Z,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,P2,2024-11-27T05:02:13.726Z,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,P3,2024-11-27T05:02:13.726Z,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,P4,2024-11-27T05:02:13.726Z,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,P5,2024-11-27T05:02:13.726Z,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [26]:
# Ajusta os tipos de dados de forma automática
df_dsa_produtos = df_dsa_produtos.convert_dtypes(infer_objects=True, convert_boolean=False)

In [27]:
# Ajusta o tipo de dado como string de forma explícita
df_dsa_produtos['id_produto'] = df_dsa_produtos['id_produto'].astype('string')
df_dsa_produtos['event_time'] = df_dsa_produtos['event_time'].astype('string')

In [28]:
df_dsa_produtos.dtypes

id_produto                       string[python]
event_time                       string[python]
category_baby_food_formula                Int64
category_baking_ingredients               Int64
category_candy_chocolate                  Int64
category_chips_pretzels                   Int64
category_cleaning_products                Int64
category_coffee                           Int64
category_cookies_cakes                    Int64
category_crackers                         Int64
category_energy_granola_bars              Int64
category_frozen_meals                     Int64
category_hair_care                        Int64
category_ice_cream_ice                    Int64
category_juice_nectars                    Int64
category_packaged_cheese                  Int64
category_refrigerated                     Int64
category_soup_broth_bouillon              Int64
category_spices_seasonings                Int64
category_tea                              Int64
category_vitamins_supplements           

In [29]:
df_dsa_produtos.head()

,id_produto,event_time,category_baby_food_formula,category_baking_ingredients,category_candy_chocolate,category_chips_pretzels,category_cleaning_products,category_coffee,category_cookies_cakes,category_crackers,...,category_hair_care,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt
0,P1,2024-11-27T05:02:13.726Z,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,P2,2024-11-27T05:02:13.726Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,P3,2024-11-27T05:02:13.726Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,P4,2024-11-27T05:02:13.726Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,P5,2024-11-27T05:02:13.726Z,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [30]:
# Carrega o dataset de pedidos
df_dsa_pedidos = pd.read_csv('dados/pedidos.csv')

In [31]:
df_dsa_pedidos.head()

,id_pedido,id_cliente,id_produto,quantidade,novo_pedido,event_time,num_dias_ativos_des_ultima_compra
0,O1,C5731,P16,0.913465,1,2024-11-27T05:02:16.934Z,0.201550
1,O2,C3541,P12802,0.663168,1,2024-11-27T05:02:16.934Z,0.575581
2,O3,C7402,P8320,0.629604,1,2024-11-27T05:02:16.934Z,0.073643
3,O4,C7356,P5165,0.618911,0,2024-11-27T05:02:16.934Z,0.552326
4,O5,C5806,P12940,0.053168,1,2024-11-27T05:02:16.934Z,0.616279


In [32]:
# Ajusta os tipos de dados de forma automática
df_dsa_pedidos = df_dsa_pedidos.convert_dtypes(infer_objects=True, convert_boolean=False)

In [33]:
# Ajusta o tipo de dado como string de forma explícita
df_dsa_pedidos['id_pedido'] = df_dsa_pedidos['id_pedido'].astype('string')
df_dsa_pedidos['id_cliente'] = df_dsa_pedidos['id_cliente'].astype('string')
df_dsa_pedidos['id_produto'] = df_dsa_pedidos['id_produto'].astype('string')
df_dsa_pedidos['event_time'] = df_dsa_pedidos['event_time'].astype('string')

In [34]:
df_dsa_pedidos.dtypes

id_pedido                            string[python]
id_cliente                           string[python]
id_produto                           string[python]
quantidade                                  Float64
novo_pedido                                   Int64
event_time                           string[python]
num_dias_ativos_des_ultima_compra           Float64
dtype: object

Esse código abaixo realiza duas ações:
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
**count_clientes = df_dsa_clientes.shape[0]:**

Calcula o número de linhas no DataFrame df_dsa_clientes, o que corresponde à quantidade de registros (ou clientes, neste caso). A propriedade shape retorna uma tupla (n_linhas, n_colunas) e o índice [0] acessa o número de linhas.

**%store count_clientes:**

Este comando é uma magic command do IPython (usado, por exemplo, no Jupyter Notebook) que armazena a variável count_clientes no namespace persistente do IPython. Isso permite que o valor de customers_count esteja disponível em outras células ou sessões do mesmo ambiente, mesmo após reinicializações.

In [35]:
count_clientes = df_dsa_clientes.shape[0]
%store count_clientes

Stored 'count_clientes' (int)


In [36]:
count_produtos = df_dsa_produtos.shape[0]
%store count_produtos

Stored 'count_produtos' (int)


In [37]:
count_pedidos = df_dsa_pedidos.shape[0]
%store count_pedidos

Stored 'count_pedidos' (int)


## Criando os Grupos de Recursos (Feature Groups)
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [38]:
# Captura a data corrente do sistema
current_timestamp = strftime('%m-%d-%H-%M', gmtime())

In [39]:
# Prefixo para rastrear todos os grupos de recursos criados
fs_prefix = 'fsdsa-' 

In [40]:
clientes_feature_group_name = f'{fs_prefix}clientes-{current_timestamp}'
%store clientes_feature_group_name
produtos_feature_group_name = f'{fs_prefix}produtos-{current_timestamp}'
%store produtos_feature_group_name
pedidos_feature_group_name = f'{fs_prefix}pedidos-{current_timestamp}'
%store pedidos_feature_group_name

Stored 'clientes_feature_group_name' (str)
Stored 'produtos_feature_group_name' (str)
Stored 'pedidos_feature_group_name' (str)


In [41]:
logger.info(f'Nome do Grupos de Recursos de Clientes = {clientes_feature_group_name}')
logger.info(f'Nome do Grupos de Recursos de Produtos = {produtos_feature_group_name}')
logger.info(f'Nome do Grupos de Recursos de Pedidos = {pedidos_feature_group_name}')

Nome do Grupos de Recursos de Clientes = fsdsa-clientes-11-28-21-30
Nome do Grupos de Recursos de Produtos = fsdsa-produtos-11-28-21-30
Nome do Grupos de Recursos de Pedidos = fsdsa-pedidos-11-28-21-30


Essa célula de código abaixo cria instâncias de FeatureGroup, que representam grupos de features organizadas no Amazon SageMaker Feature Store. Cada grupo é associado a um conjunto específico de dados ou entidade, como clientes, produtos ou pedidos.

In [42]:
# Cria os grupos
clientes_feature_group = FeatureGroup(name = clientes_feature_group_name, sagemaker_session = sagemaker_session)
produtos_feature_group = FeatureGroup(name = produtos_feature_group_name, sagemaker_session = sagemaker_session)
pedidos_feature_group = FeatureGroup(name = pedidos_feature_group_name, sagemaker_session = sagemaker_session)

## Criando as Definições de Features Para os Grupos de Recursos

As linhas abaixo carregam as definições de features para cada Feature Group com base no DataFrame.

**load_feature_definitions:**

Esse método analisa o DataFrame fornecido (df_dsa_clientes) para identificar automaticamente as colunas e seus tipos de dados. A partir disso, ele define as features (colunas) que pertencem ao grupo clientes_feature_group e seus tipos de dados associados (ex.: String, Int64, Float64).

Esse comando configura o esquema do Feature Group, incluindo:

- Nomes das features (baseados nos nomes das colunas do DataFrame).
- Tipos de dados de cada feature.

Essa etapa é essencial para preparar o Feature Group antes de ingerir dados reais, garantindo que o armazenamento e o processamento das features no SageMaker Feature Store sejam consistentes com o DataFrame fornecido.

In [43]:
# Carrega definições dos atributos
clientes_feature_group.load_feature_definitions(data_frame = df_dsa_clientes)

[FeatureDefinition(feature_name='id_cliente', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='genero', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='casado', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='idade_18-29', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='idade_30-39', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='idade_40-49', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='idade_50-59', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='ida

In [44]:
# Carrega definições dos atributos
produtos_feature_group.load_feature_definitions(data_frame = df_dsa_produtos)

[FeatureDefinition(feature_name='id_produto', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='category_baby_food_formula', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='category_baking_ingredients', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='category_candy_chocolate', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='category_chips_pretzels', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='category_cleaning_products', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='category_coffee', feature_type=<FeatureTypeEn

In [45]:
# Carrega definições dos atributos
pedidos_feature_group.load_feature_definitions(data_frame = df_dsa_pedidos)

[FeatureDefinition(feature_name='id_pedido', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='id_cliente', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='id_produto', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='quantidade', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='novo_pedido', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='event_time', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='num_dias_ativos_des_ultima_compra', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None)]

## Armazenamento Offline da Feature Store - Apache Iceberg x AWS Glue

O Amazon SageMaker Feature Store oferece suporte aos formatos de tabela AWS Glue e Apache Iceberg para o armazenamento offline. Você pode escolher o formato de tabela ao criar um novo grupo de recursos.

Neste notebook, usaremos o formato de tabela Iceberg. Usar o Apache Iceberg para armazenar recursos acelera o desenvolvimento do modelo ao permitir um desempenho de consulta mais rápido ao extrair conjuntos de dados de treinamento de ML, aproveitando a compactação da tabela Iceberg. Dependendo do design dos seus grupos de recursos e sua escala, você pode experimentar melhorias de desempenho de consulta de treinamento de 10x a 100x usando esse novo recurso.

Se precisar usar o formato de tabela Glue, atualize a variável abaixo para `'Glue'`. Para obter mais informações sobre formatos de armazenamento offline, consulte a [documentação](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-offline.html).

In [46]:
# Define o formato das tabelas
table_format_param = 'ICEBERG'

In [47]:
# Pode ser ajustado com Iceberg ou Glue
if table_format_param == 'ICEBERG':
    table_format = TableFormatEnum.ICEBERG
else:
    table_format = TableFormatEnum.GLUE

## Usando Apache Iceberg Para o Armazenamento Offline da Feature Store

In [48]:
# Define a função para armazenar um Feature Group
def dsa_armazena_feature_group(feature_group):
    
    # Obtém o status inicial do Feature Group a partir de sua descrição
    status = feature_group.describe().get('FeatureGroupStatus')
    
    # Exibe o status inicial no console
    print(f'Status: {status}')
    
    # Enquanto o status for "Creating", continua aguardando
    while status == 'Creating':
        
        # Registra no log que está aguardando a criação do Feature Group
        logger.info(f'Esperando pelo feature group: {feature_group.name} ser criado...')
        
        # Aguarda 5 segundos antes de verificar o status novamente
        time.sleep(5)
        
        # Atualiza o status do Feature Group
        status = feature_group.describe().get('FeatureGroupStatus')
    
    # Se o status final não for "Created", encerra o programa com erro
    if status != 'Created':
        raise SystemExit(f'Falha ao criar o feature group {feature_group.name}: {status}')
    
    # Se o Feature Group foi criado com sucesso, registra a mensagem de sucesso no log
    logger.info(f'Feature Group {feature_group.name} foi criado com sucesso.')

In [49]:
# Cria o repositório de armazenamento do feature group
clientes_feature_group.create(s3_uri = f's3://{default_bucket}/{prefix}', 
                              record_identifier_name = 'id_cliente', 
                              event_time_feature_name = 'event_time', 
                              role_arn = role, 
                              enable_online_store = True,
                              table_format = table_format)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:890582101704:feature-group/fsdsa-clientes-11-28-21-30',
 'ResponseMetadata': {'RequestId': 'd7bffb8a-b612-4756-9f69-a987ebca30cc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd7bffb8a-b612-4756-9f69-a987ebca30cc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Thu, 28 Nov 2024 22:39:28 GMT'},
  'RetryAttempts': 0}}

In [50]:
# Verifica o status de armazenamento
dsa_armazena_feature_group(clientes_feature_group)

Esperando pelo feature group: fsdsa-clientes-11-28-21-30 ser criado...


Status: Creating


Esperando pelo feature group: fsdsa-clientes-11-28-21-30 ser criado...
Esperando pelo feature group: fsdsa-clientes-11-28-21-30 ser criado...
Feature Group fsdsa-clientes-11-28-21-30 foi criado com sucesso.


In [51]:
# Cria o repositório de armazenamento do feature group
produtos_feature_group.create(s3_uri = f's3://{default_bucket}/{prefix}', 
                              record_identifier_name = 'id_produto', 
                              event_time_feature_name = 'event_time', 
                              role_arn = role, 
                              enable_online_store = True,
                              table_format = TableFormatEnum.ICEBERG)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:890582101704:feature-group/fsdsa-produtos-11-28-21-30',
 'ResponseMetadata': {'RequestId': '20679c6e-4d13-429c-be35-7bfed73dd565',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '20679c6e-4d13-429c-be35-7bfed73dd565',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Thu, 28 Nov 2024 22:46:33 GMT'},
  'RetryAttempts': 0}}

In [52]:
# Verifica o status de armazenamento
dsa_armazena_feature_group(produtos_feature_group)

Esperando pelo feature group: fsdsa-produtos-11-28-21-30 ser criado...


Status: Creating


Esperando pelo feature group: fsdsa-produtos-11-28-21-30 ser criado...
Esperando pelo feature group: fsdsa-produtos-11-28-21-30 ser criado...
Esperando pelo feature group: fsdsa-produtos-11-28-21-30 ser criado...
Esperando pelo feature group: fsdsa-produtos-11-28-21-30 ser criado...
Feature Group fsdsa-produtos-11-28-21-30 foi criado com sucesso.


In [53]:
# Cria o repositório de armazenamento do feature group
pedidos_feature_group.create(s3_uri = f's3://{default_bucket}/{prefix}', 
                             record_identifier_name = 'id_pedido', 
                             event_time_feature_name = 'event_time', 
                             role_arn = role, 
                             enable_online_store = True,
                             table_format = TableFormatEnum.ICEBERG)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:890582101704:feature-group/fsdsa-pedidos-11-28-21-30',
 'ResponseMetadata': {'RequestId': 'ef06bc08-3fe7-483b-8f54-1d2057787e92',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ef06bc08-3fe7-483b-8f54-1d2057787e92',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Thu, 28 Nov 2024 22:47:31 GMT'},
  'RetryAttempts': 0}}

In [54]:
dsa_armazena_feature_group(pedidos_feature_group)

Esperando pelo feature group: fsdsa-pedidos-11-28-21-30 ser criado...


Status: Creating


Esperando pelo feature group: fsdsa-pedidos-11-28-21-30 ser criado...
Esperando pelo feature group: fsdsa-pedidos-11-28-21-30 ser criado...
Esperando pelo feature group: fsdsa-pedidos-11-28-21-30 ser criado...
Feature Group fsdsa-pedidos-11-28-21-30 foi criado com sucesso.


## Ingestão de Dados nos Grupos de Recursos

In [55]:
%%time

logger.info(f'Ingestão de Dados no Grupo de Recursos: {clientes_feature_group.name} ...')
clientes_feature_group.ingest(data_frame=df_dsa_clientes, max_processes=16, wait=True)
logger.info(f'{len(df_dsa_clientes)} registros de clientes ingeridos no grupo de recursos: {clientes_feature_group.name}')

Ingestão de Dados no Grupo de Recursos: fsdsa-clientes-11-28-21-30 ...
10000 registros de clientes ingeridos no grupo de recursos: fsdsa-clientes-11-28-21-30


CPU times: user 216 ms, sys: 219 ms, total: 435 ms
Wall time: 15.9 s


In [56]:
%%time

logger.info(f'Ingestão de Dados no Grupo de Recursos: {produtos_feature_group.name} ...')
produtos_feature_group.ingest(data_frame=df_dsa_produtos, max_processes=16, wait=True)
logger.info(f'{len(df_dsa_produtos)} registros de produtos ingeridos no grupo de recursos: {produtos_feature_group.name}')  

Ingestão de Dados no Grupo de Recursos: fsdsa-produtos-11-28-21-30 ...
17001 registros de produtos ingeridos no grupo de recursos: fsdsa-produtos-11-28-21-30


CPU times: user 297 ms, sys: 177 ms, total: 474 ms
Wall time: 24.1 s


In [57]:
%%time

logger.info(f'Ingestão de Dados no Grupo de Recursos: {pedidos_feature_group.name} ...')
pedidos_feature_group.ingest(data_frame=df_dsa_pedidos, max_processes=16, wait=True)
logger.info(f'{len(df_dsa_pedidos)} registros de pedidos ingeridos no grupo de recursos: {pedidos_feature_group.name}')

Ingestão de Dados no Grupo de Recursos: fsdsa-pedidos-11-28-21-30 ...
100000 registros de pedidos ingeridos no grupo de recursos: fsdsa-pedidos-11-28-21-30


CPU times: user 1.38 s, sys: 228 ms, total: 1.61 s
Wall time: 1min 43s


## Recuperando Dados da Feature Store Online

In [58]:
# Cria o cliente de conexão
featurestore_runtime_client = sagemaker_session.boto_session.client('sagemaker-featurestore-runtime', region_name = region)

Retrieve a record from customers feature group

In [59]:
# Aleatoriamente geramos um id de cliente
id_cliente =  f'C{randint(1, 10000)}'
logger.info(f'id_cliente = {id_cliente}') 

id_cliente = C6593


In [60]:
# Buscamos o registro associado ao id do cliente no grupo de recursos
feature_record = featurestore_runtime_client.get_record(FeatureGroupName = clientes_feature_group.name, 
                                                        RecordIdentifierValueAsString = id_cliente)

In [61]:
feature_record

{'ResponseMetadata': {'RequestId': 'f815922a-d6f0-42b1-aeaa-d509bcad3ed9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f815922a-d6f0-42b1-aeaa-d509bcad3ed9',
   'content-type': 'application/json',
   'content-length': '891',
   'date': 'Thu, 28 Nov 2024 23:28:55 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'id_cliente', 'ValueAsString': 'C6593'},
  {'FeatureName': 'genero', 'ValueAsString': '1'},
  {'FeatureName': 'casado', 'ValueAsString': '1'},
  {'FeatureName': 'event_time', 'ValueAsString': '2024-11-27T04:58:03.278Z'},
  {'FeatureName': 'idade_18-29', 'ValueAsString': '0'},
  {'FeatureName': 'idade_30-39', 'ValueAsString': '1'},
  {'FeatureName': 'idade_40-49', 'ValueAsString': '0'},
  {'FeatureName': 'idade_50-59', 'ValueAsString': '0'},
  {'FeatureName': 'idade_60-69', 'ValueAsString': '0'},
  {'FeatureName': 'idade_70-plus', 'ValueAsString': '0'},
  {'FeatureName': 'num_dias_ativo', 'ValueAsString': '0.2924657534246575'}]}

## Listando os Grupos de Recursos

In [62]:
# Inicializa o cliente do SageMaker
sagemaker_client = boto3.client('sagemaker')

# Lista os Feature Groups
response = sagemaker_client.list_feature_groups()

# Itera sobre os Feature Groups e exibe os nomes
feature_groups = response.get('FeatureGroupSummaries', [])
for fg in feature_groups:
    print(f"Feature Group Name: {fg['FeatureGroupName']}")

Feature Group Name: fsdsa-produtos-11-28-21-30
Feature Group Name: fsdsa-pedidos-11-28-21-30
Feature Group Name: fsdsa-clientes-11-28-21-30


## Deletando os Grupos de Recursos de Forma Automática

In [63]:
# Inicializa o cliente do SageMaker e a sessão do SageMaker
sagemaker_client = boto3.client('sagemaker')
from sagemaker.session import Session
sagemaker_session = Session()

# Lista os Feature Groups
response = sagemaker_client.list_feature_groups()
feature_groups = response.get('FeatureGroupSummaries', [])

# Itera sobre os Feature Groups e os deleta
for fg in feature_groups:
    feature_group_name = fg['FeatureGroupName']
    feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)
    try:
        feature_group.delete()
        print(f"Feature Group '{feature_group_name}' deletado com sucesso.")
    except Exception as e:
        print(f"Erro ao deletar o Feature Group '{feature_group_name}': {str(e)}")

Feature Group 'fsdsa-produtos-11-28-21-30' deletado com sucesso.
Feature Group 'fsdsa-pedidos-11-28-21-30' deletado com sucesso.
Feature Group 'fsdsa-clientes-11-28-21-30' deletado com sucesso.


# Fim